In [ ]:
model.load_state_dict(torch.load("my_ae_trained.pth"))
torch.save(ae.state_dict(), "my_ae_trained.pth")
#torch.save(model.state_dict(),"./mount/My Drive/Colab Notebooks/GDLgon/my_vae_celeb_trained.pth")

vae.load_state_dict(torch.load("my_vae_trained.pth",map_location=torch.device('cpu')))

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch

class MyDataset(Dataset):
    def __init__(self):
        self.x_input = np.load("MNIST_x_train.npy").astype(np.float32).reshape(-1, 28*28)

    def __len__(self):
        return len(self.x_input)

    def __getitem__(self, idx):
        return torch.cuda.FloatTensor(self.x_input[idx])

data_loader = DataLoader(MyDataset(), batch_size=1000, shuffle=True,\
    generator=torch.Generator(device='cuda'))

In [ ]:
from google.colab import drive
drive.mount('./mount')

In [ ]:
import torch
if torch.cuda.is_available():
  torch.set_default_tensor_type(torch.cuda.FloatTensor)
  print("using cuda:", torch.cuda.get_device_name(0))
  pass

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
import numpy as np

class MnistDataset(Dataset):
    train_data = pd.read_csv("./mount/My Drive/Colab Notebooks/firstGAN/mnist_train.csv")
    train_data =train_data.to_numpy()
    # read csv
    def __init__(self):
        super().__init__()
        
    def __len__(self):
        return len(MnistDataset.train_data)
    def __getitem__(self, index):
        #return torch.tensor(MnistDataset.train_data[index,1:]/255,dtype=torch.float32)
        return torch.cuda.FloatTensor(MnistDataset.train_data[index,1:]/255.0)

    def plot_image(self,n):
        data = MnistDataset.train_data[n,1:].reshape(28,28)
        plt.figure(figsize=(3,3))
        plt.imshow(data)



In [ ]:
print(torch.cuda.memory_allocated(device) / (1024*1024*1024))
print(torch.cuda.max_memory_allocated(device) / (1024*1024*1024))
print(torch.cuda.memory_summary(device, abbreviated=True))

In [ ]:
import torch
import numpy 
from torch.nn import MSELoss
from torch.optim import Adam

class VAE(torch.nn.Module):
    def __init__(self,dim_latent = 2):
        super().__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(784,128),
            torch.nn.ReLU(),
            torch.nn.Linear(128,64),
            torch.nn.ReLU(),
            torch.nn.Linear(64,2*dim_latent),
        )
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(dim_latent,64),
            torch.nn.ReLU(),
            torch.nn.Linear(64,128),
            torch.nn.ReLU(),
            torch.nn.Linear(128,784),
        )
        self.dim_latent = dim_latent
        self.optim_en = Adam(self.encoder.parameters(),lr=0.02)
        self.optim_de = Adam(self.decoder.parameters(),lr=0.02)
        self.loss_fn = MSELoss()

        self.num_epoch=400
        self.hist_loss = []
        self.hist_model = []

    def forward(self,x):
        x = self.encoder(x).view(-1,2,self.dim_latent)

        self.mu = x[:, 0, :]
        self.sigma = torch.exp(x[:, 1, :])
        self.z = self.mu + self.sigma * torch.randn_like(self.mu)

        de = self.decoder(self.z)
        return de

    def train(self,x_input):
        for epoch in range(1, 401):
            self.optim_en.zero_grad()
            self.optim_de.zero_grad()
            y_pred = self.forward(x_input)
            
            mse_loss = ((x_input - y_pred)**2).sum()
            kl_loss = (self.sigma**2 + self.mu**2 - torch.log(self.sigma) - 0.5).sum()
            loss = mse_loss + kl_loss

            loss.backward()
            self.optim_de.step()
            self.optim_en.step()
            if epoch % 50 == 0:
                print(f"Epoch {epoch}: loss = {loss.item()}")

    def visualize(self):
        pass
        